## Youtube Scrape and Check Youtube Ads

In [64]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests



In [65]:
# ID Video YouTube
# https://www.youtube.com/watch?v=o0oW3lPoOXM
youtubeid = "i6IOiUi6IYY"  # Ganti dengan ID video yang diinginkan

In [66]:
try:
    response = requests.get("https://www.youtube.com")
    if response.status_code == 200:
        print("Koneksi ke YouTube berhasil.")
    else:
        print("Koneksi ke YouTube tidak berhasil.")
except requests.ConnectionError as e:
    print("Gagal menghubungi YouTube:", e)

Koneksi ke YouTube berhasil.


In [67]:

# Path ke driver Chrome
# driver_path = '/path/to/chromedriver'  # Ganti dengan path chromedriver Anda
profile_path = r'C:\Users\LENOVO\AppData\Local\Google\Chrome\User Data\Default'  # Ganti dengan path profil Chrome Anda

def initialize_driver_with_profile(profile_path):
    """Inisialisasi driver dengan profil Chrome yang sudah ada."""
    chrome_options = Options()
    chrome_options.add_argument(f"user-data-dir={profile_path}")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-first-run")
    chrome_options.add_argument("--remote-debugging-port=9222")
    chrome_options.add_argument("--enable-logging")
    chrome_options.add_argument("--v=1")  # Level logging dasar
    # service = Service(driver_path)
    # driver = webdriver.Chrome(service=service, options=chrome_options)
    driver = webdriver.Chrome(options=chrome_options)
    return driver


In [68]:
def detect_and_skip_ad(driver):
    """Periksa iklan dan klik tombol 'Skip Ad' saat tersedia."""
    ad_present = False
    try:
        # Cek jika elemen iklan atau teks sponsor ada
        WebDriverWait(driver, 15).until(
            EC.presence_of_any_elements_located([
                (By.CLASS_NAME, "video-ads.ytp-ad-module"),               # Div dengan kelas iklan
                (By.CLASS_NAME, "ytp-ad-player-overlay-layout_")          # Overlay iklan
            ])
        )
        ad_present = True
        print("Iklan terdeteksi.")

        # Jika iklan terdeteksi, tunggu tombol "Skip Ad" dan klik jika ada
        skip_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "ytp-skip-ad-button"))
        )
        skip_button.click()
        print("Tombol 'Skip Ad' diklik.")
        
    except Exception as e:
        print("Tidak ada iklan atau tombol 'Skip Ad' tidak ditemukan.")
        print(f"Error: {e}")

    return ad_present

In [69]:
def get_all_youtube_comments(driver, max_comments=None):
    """Mengambil semua komentar dari video YouTube dengan scroll tanpa batas."""
    comments = []
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        # Scroll ke bawah untuk memuat komentar baru
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)  # Jeda untuk memuat komentar baru

        # Ambil elemen komentar
        comment_elements = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
        for element in comment_elements[len(comments):]:  # Mulai dari komentar baru
            comment_text = element.text
            if comment_text:
                comments.append(comment_text)
                # Batas maksimal komentar jika disetel
                if max_comments and len(comments) >= max_comments:
                    return comments

        # Cek tinggi baru halaman untuk melihat apakah ada yang baru dimuat
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:  # Berhenti jika tidak ada lagi yang dimuat
            break
        last_height = new_height

    return comments

In [70]:
import pandas as pd
def save_to_csv(comments, filename):
    """Menyimpan komentar ke file CSV."""
    df = pd.DataFrame(comments, columns=["Comment"])
    df.to_csv(filename, index=False)
    print(f"Komentar berhasil disimpan di {filename}")

In [71]:

video_url = f"https://www.youtube.com/watch?v={youtubeid}"
# URL video YouTube
# video_url = "https://www.youtube.com/watch?v=i6IOiUi6IYY"  # Ganti dengan ID video yang diinginkan

# Inisialisasi WebDriver dengan profil Chrome
driver = initialize_driver_with_profile(profile_path)

# Akses URL video YouTube
driver.get(video_url)
time.sleep(5)  # Tunggu halaman dimuat

# Periksa iklan dan lewati jika ada
ad_present = detect_and_skip_ad(driver)


# Jika tidak ada iklan atau iklan sudah dilewati, mulai mengambil komentar
if not ad_present or ad_present:
    comments = get_all_youtube_comments(driver, max_comments=None)  # None untuk scroll tanpa batas
    save_to_csv(comments, filename=f"youtube_comments_{youtubeid}.csv")


Tidak ada iklan atau tombol 'Skip Ad' tidak ditemukan.
Error: module 'selenium.webdriver.support.expected_conditions' has no attribute 'presence_of_any_elements_located'
Komentar berhasil disimpan di youtube_comments_i6IOiUi6IYY.csv


In [75]:
comments

['Saka is the best he prove this  everytime by doing goal in every important match',
 'Thomas Partey… man has been bringing it lately.',
 'Saka and Partey were so so good man ',
 'That pass from Ben White to Saka was amazing. ',
 'Bukayo Saka, i could watch you play football for hours and not get bored. What a player man we are so lucky to have him',
 "We are nearing perfection that's all I can say .Top quality performance from Saka and the team",
 'Not even a Liverpool fan but that play from Nunez and Salah ',
 'Van Dijk is just always there. He has so much presence',
 'I’m so happy for salah to stay at Liverpool. Magic player.',
 '" Keep Saka\'s name out your mouth,  dude is world class.',
 "From a Liverpool fan, We didn't have Allison, but I think those goals go in anyway, there's so much quality in this Arsenal side, I actually expected a loss, You lot are actually so good you drag our performances up with you, Especially Nunez...",
 '1:23 Trent long pass moves like a golf ball, Nu

In [73]:

# print(comments)
save_to_csv(comments, filename=f"youtube_comments_{youtubeid}.csv")
# Tutup driver setelah selesai
driver.quit()

['Saka is the best he prove this  everytime by doing goal in every important match', 'Thomas Partey… man has been bringing it lately.', 'Saka and Partey were so so good man ', 'That pass from Ben White to Saka was amazing. ', 'Bukayo Saka, i could watch you play football for hours and not get bored. What a player man we are so lucky to have him', "We are nearing perfection that's all I can say .Top quality performance from Saka and the team", 'Not even a Liverpool fan but that play from Nunez and Salah ', 'Van Dijk is just always there. He has so much presence', 'I’m so happy for salah to stay at Liverpool. Magic player.', '" Keep Saka\'s name out your mouth,  dude is world class.', "From a Liverpool fan, We didn't have Allison, but I think those goals go in anyway, there's so much quality in this Arsenal side, I actually expected a loss, You lot are actually so good you drag our performances up with you, Especially Nunez...", '1:23 Trent long pass moves like a golf ball, Nunez didnt e